In [78]:
import numpy as np
import random
from itertools import product
import pandas as pd

class Frota:
    def __init__(self, Periodo, TamanhoFrota,Demanda):
        self.ProbQuebra_0 = 0.01
        self.Depreciacao = 0.4
        self.ProbPar = 0.4
        # self.Parametros = {}
        self.Periodo = Periodo
        self.TamanhoFrota = TamanhoFrota
        self.Demanda = Demanda
        self.ProbQuebraFcn = lambda x: self.ProbQuebra_0/(self.ProbQuebra_0+(1-self.ProbQuebra_0)*np.exp(-self.ProbPar*x))
        self.EstadosAcoes = {}
        # self.Acoes = {}
        self.Politica = []
        self.ConstroeEstadosAcoes()


    def ConstroeEstadosAcoes(self):
        aux = []
        for i in range(self.Periodo+1):
            for j in range(i+1):
                aux.append((i,j))
        
        estados = list(product(*[aux for i in range(self.TamanhoFrota)]))
        
        acoes = list(product(*[[0,1,2] for i in range(self.TamanhoFrota)]))
        self.Estados = estados
        # self.Acoes = acoes
        estado_acoes = {} 
        for est in estados:
            vmax = max([e[0] for e in est])
            for i in range(vmax,self.Periodo):
                aux = []
                # estado_acoes.append((*est,i))
                for ac in acoes:
                    cnt = sum([a==0 for a in ac])
                    if cnt>= self.Demanda[i]:
                        if i<self.Periodo-1:
                            aux.append(ac)
                        else:
                            aux.append(tuple([-1]*self.TamanhoFrota))
                estado_acoes[(*est,i)]=aux
        estado_acoes[(tuple([(-1,-1)]*self.TamanhoFrota),i+1)]=[tuple([-1]*self.TamanhoFrota)]
        self.EstadosAcoes=estado_acoes
        return estado_acoes 
    
    def Recompensa(self,est):
        tvida = est[0]

        # define o valor do equipamento como função da idade do equipamento
        valor_equipamento = np.exp(-0.005*tvida)

        # define o valor da manutenção
        valor_manutencao = 0.02 - 0.01*np.exp(-0.1*tvida)
        
        # define o valor da manutenção se der defeito
        valor_defeito = -2*valor_manutencao/valor_equipamento
        
        # Cálculo das recompensas SEM defeito e COM defeito
        res_a0_op,res_a0_de = 0.01/valor_equipamento,valor_defeito

        # Cálculo das recompensas na manutenção
        res_a1 = -valor_manutencao/valor_equipamento
        
        # Cálculo das recompensas na troca de equipamento
        res_a2 = -(1 - 0.8*valor_equipamento)
        
        res = [res_a0_op,res_a0_de,res_a1,res_a2]
        if est[0]==-1:
            res = [0,0,0,0]
        return res

    def ProximoEstado(self,estado):
        acoes = self.EstadosAcoes[estado]
        NovosEstados = {}
        for ac in acoes:
            NovoEstado = [[]]*self.TamanhoFrota
            Recompensa = [[]]*self.TamanhoFrota
            Probabilidades = [[]]*self.TamanhoFrota
            for i,a in enumerate(ac):
                rcp_aux = self.Recompensa(estado[i])
                if a==0:
                    NovoEstado[i] =[(estado[i][0]+1,estado[i][1]+1),(estado[i][0]+1,0)]
                    prob_quebrar = self.ProbQuebraFcn(estado[i][1])
                    Probabilidades[i] = [1-prob_quebrar,prob_quebrar]
                    Recompensa[i] = [rcp_aux[0],rcp_aux[1]]
                if a==1:
                    NovoEstado[i] = [(estado[i][0]+1,0)]
                    Probabilidades[i] = [1]
                    Recompensa[i] = [rcp_aux[2]]
                if a==2:
                    NovoEstado[i] = [(0,0)]
                    Probabilidades[i] = [1]
                    Recompensa[i] = [rcp_aux[3]]
                if a==-1:
                    NovoEstado[i] = [(-1,-1)]
                    Probabilidades[i] = [1]
                    Recompensa[i] = [0]

            # print(NovoEstado)

            probs = list(product(*Probabilidades))
            recs = list(product(*Recompensa))
            probs = [np.prod(pr) for pr in probs]
            recs = [np.sum(rc) for rc in recs]
            NovosEstados["estados"]=[(*x,estado[-1]+1) for x in list(product(*NovoEstado))]
            NovosEstados["recompensas"]=recs
            NovosEstados["probabilidades"]=probs

        return NovosEstados




In [79]:
mdl = Frota(3,2,[2,1,2])

In [80]:
mdl.EstadosAcoes

{((0, 0), (0, 0), 0): [(0, 0)],
 ((0, 0), (0, 0), 1): [(0, 0), (0, 1), (0, 2), (1, 0), (2, 0)],
 ((0, 0), (0, 0), 2): [(-1, -1)],
 ((0, 0), (1, 0), 1): [(0, 0), (0, 1), (0, 2), (1, 0), (2, 0)],
 ((0, 0), (1, 0), 2): [(-1, -1)],
 ((0, 0), (1, 1), 1): [(0, 0), (0, 1), (0, 2), (1, 0), (2, 0)],
 ((0, 0), (1, 1), 2): [(-1, -1)],
 ((0, 0), (2, 0), 2): [(-1, -1)],
 ((0, 0), (2, 1), 2): [(-1, -1)],
 ((0, 0), (2, 2), 2): [(-1, -1)],
 ((1, 0), (0, 0), 1): [(0, 0), (0, 1), (0, 2), (1, 0), (2, 0)],
 ((1, 0), (0, 0), 2): [(-1, -1)],
 ((1, 0), (1, 0), 1): [(0, 0), (0, 1), (0, 2), (1, 0), (2, 0)],
 ((1, 0), (1, 0), 2): [(-1, -1)],
 ((1, 0), (1, 1), 1): [(0, 0), (0, 1), (0, 2), (1, 0), (2, 0)],
 ((1, 0), (1, 1), 2): [(-1, -1)],
 ((1, 0), (2, 0), 2): [(-1, -1)],
 ((1, 0), (2, 1), 2): [(-1, -1)],
 ((1, 0), (2, 2), 2): [(-1, -1)],
 ((1, 1), (0, 0), 1): [(0, 0), (0, 1), (0, 2), (1, 0), (2, 0)],
 ((1, 1), (0, 0), 2): [(-1, -1)],
 ((1, 1), (1, 0), 1): [(0, 0), (0, 1), (0, 2), (1, 0), (2, 0)],
 ((1, 1), (1, 

In [67]:
res = mdl.ProximoEstado(((0, 0), (0, 0), 0))

In [68]:
res

{'estados': [((1, 1), (1, 1), 1),
  ((1, 1), (1, 0), 1),
  ((1, 0), (1, 1), 1),
  ((1, 0), (1, 0), 1)],
 'recompensas': [0.02, -0.01, -0.01, -0.04],
 'probabilidades': [0.9801, 0.0099, 0.0099, 0.0001]}